In [137]:
import wsitools
import importlib
importlib.reload(wsitools)

<module 'wsitools' from 'D:\\Programming\\wsi-tools\\wsitools.py'>

In [138]:
wsitools.napari_tile_inspector(zarr_path = Path(r"D:\Programming\whole_slide_analysis\Images_raw_regions_melanoma\region_001"))

Viewer(camera=Camera(center=(0.0, 511.5, 511.5), zoom=0.8804199218750001, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True, orientation=(<DepthAxisOrientation.TOWARDS: 'towards'>, <VerticalAxisOrientation.DOWN: 'down'>, <HorizontalAxisOrientation.RIGHT: 'right'>)), cursor=Cursor(position=(1.0, 1.0), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=2, ndisplay=2, order=(0, 1), axis_labels=('0', '1'), rollable=(True, True), range=(RangeTuple(start=0.0, stop=1023.0, step=1.0), RangeTuple(start=0.0, stop=1023.0, step=1.0)), margin_left=(0.0, 0.0), margin_right=(0.0, 0.0), point=(511.0, 511.0), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False, spacing=0.0), layers=[<Image layer 'KSS DAPI' at 0x2078c2279b0>, <Image layer 'KSS TXRed' at 0x2078cd39700>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[]

INFO: Applying: [{'method': 'rolling_ball', 'sigma_px': 50, 'channels': [0]}]. Axis order: CYX.
INFO: Applied rolling_ball to ['KSS TXRed']
INFO: Applying: [{'method': 'rolling_ball', 'sigma_px': 50, 'channels': [0]}]. Axis order: CYX.
INFO: Applied rolling_ball to ['KSS TXRed']


In [97]:
channel_names = ['a','b','c']

In [98]:
list(zip(channel_names, range(len(channel_names))))

[('a', 0), ('b', 1), ('c', 2)]

In [18]:
import os
from ome_types import from_xml
from pathlib import Path
import dask
import dask.array as da
import zarr

In [120]:
zarr_path = Path(r"D:\Programming\whole_slide_analysis\Images_raw_regions_brain\region_001")

In [121]:
# Open raw folder as Zarr
store = zarr.open(zarr_path, mode='r+')['0']

# Access specific level of pyramid (usually top, 0)
raw_zarr = store['0']

raw_dask = da.from_zarr(raw_zarr)

In [122]:
ndim = raw_dask.ndim

In [123]:
ndim = raw_dask.ndim
σ = 50
sigma = (0.0,)*(ndim-2)+(σ,σ)

In [124]:
sigma

(0.0, 0.0, 0.0, 50, 50)

In [125]:
raw_dask

dask.array<from-zarr, shape=(1, 3, 1, 17358, 27903), dtype=>u2, chunksize=(1, 1, 1, 1024, 512), chunktype=numpy.ndarray>

In [128]:
# Load metadata
with open(os.path.join(zarr_path, "OME", "METADATA.ome.xml"), encoding='utf-8') as f:
    xml_text = f.read()

xml_text = xml_text.replace("Âµm", "µm")

# Now parse with ome-types
ome = from_xml(xml_text)
    
pixels = ome.images[0].pixels
size_z, size_c, size_y, size_x = pixels.size_z, pixels.size_c, pixels.size_y, pixels.size_x

In [129]:
ome.images[0].pixels.channels

[Channel(
    id='Channel:2:0',
    name='KSS FITC',
    detector_settings={'id': 'Detector:0:2', 'binning': <Binning.ONEBYONE: '1x1'>},
    light_path={},
    samples_per_pixel=1,
    emission_wavelength=518.0,
 ),
 Channel(
    id='Channel:2:1',
    name='KSS Cy3',
    detector_settings={'id': 'Detector:0:2', 'binning': <Binning.ONEBYONE: '1x1'>},
    light_path={},
    samples_per_pixel=1,
    emission_wavelength=565.0,
 ),
 Channel(
    id='Channel:2:2',
    name='KSS Cy5',
    detector_settings={'id': 'Detector:0:2', 'binning': <Binning.ONEBYONE: '1x1'>},
    light_path={},
    samples_per_pixel=1,
    emission_wavelength=670.0,
 )]

In [130]:
# Extract channels
channels = ome.images[0].pixels.channels

# List of channel names
channel_names = [ch.name for ch in channels]

# Helper: map emission wavelength to a rough color name
def wavelength_to_color(wavelength_nm):
    if wavelength_nm < 500:
        return 'blue'
    elif 500 <= wavelength_nm < 570:
        return 'green'
    elif 570 <= wavelength_nm < 590:
        return 'yellow'
    elif 590 <= wavelength_nm < 620:
        return 'orange'
    elif 620 <= wavelength_nm < 750:
        return 'red'
    else:
        return 'gray'  # fallback for IR/UV or unknown

# List of colors for napari visualization
channel_colors = [wavelength_to_color(ch.emission_wavelength) for ch in channels]


In [14]:
channel_names

['KSS DAPI', 'KSS TXRed']